In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
import urllib.request
import webbrowser
import os
import pandas as pd
from datetime import datetime, timedelta
import urllib.request

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

ModuleNotFoundError: ignored

In [ ]:
#전처리 부분

#파일 받아오기
file = "full_videos_list.csv"
df=pd.read_csv(file)

#컬럼명 지정
df.columns=['a','team','site']

#team컬럼에 [다시보기]제거
df['team']=df['team'].str.replace('\[다시보기\] ','')

#필요없는 컬럼 제거
df.drop(['a'],axis=1,inplace=True)

#dh(더블헤더)컬럼 생성
def read_dh(s):
  if 'DH1' in s:
    return '1'
  elif 'DH2' in s:
    return '2'
  return '0' 

df['dh']=df['team'].apply(lambda x:read_dh(x))

#team컬럼 형식 통일
df['team']=df['team'].str.replace('월_','월 ')
df['team']=df['team'].str.replace('일 ','월 ')

#team컬럼 '_'기준으로 나눠서 날짜, 파일번호, 팀 컬럼 생성
df['date']=df['team'].str.split('_').str[1]
df['number']=df['team'].str.split('_').str[2]
df['team']=df['team'].str.split('_').str[0]

#date컬럼 형식 통일
df['date']=df['date'].str.replace(' (DH1)','')

#number컬럼 형식 통일
df['number']=df['number'].str.replace('부','')

#team컬럼 형식을 예) 롯데 vs 삼성 으로 되있는걸 LTSS 이런 형식으로 통일
df['team']=df['team'].str.replace(' vs ','')
df['team']=df['team'].str.replace('롯데','LT')
df['team']=df['team'].str.replace('KIA','HT')
df['team']=df['team'].str.replace('키움','WO')
df['team']=df['team'].str.replace('두산','OB')
df['team']=df['team'].str.replace('SSG','SK')
df['team']=df['team'].str.replace('삼성','SS')
df['team']=df['team'].str.replace('한화','HH')

#team컬럼 필요없는 부분 제거
df['team']=df['team'].str[:4]

#날짜 형식 통일
df['mon']=df['date'].str.split('월 ').str[0]
df['day']=df['date'].str.split('월 ').str[1]
df['day']=df['day'].str.replace('일','')

def change_date(s):
    if len(s)==1:
        return str('0'+s)
    else:
        return str(s)
    
df['mon']=df['mon'].apply(lambda x:change_date(x))
df['day']=df['day'].apply(lambda x:change_date(x))
df['date']='2021'+df['mon']+df['day']
df.drop(['mon','day'],axis=1,inplace=True)

<ipython-input-112-4bf360bee205>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['team']=df['team'].str.replace('\[다시보기\] ','')
<ipython-input-112-4bf360bee205>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['date']=df['date'].str.replace(' (DH1)','')


In [ ]:
#driver생성
driver = webdriver.Chrome("c:/data/chromedriver")#, options=chrome_options)

#src가 블롭형식으로 되어있을 때 그 리스트를 받기 위해서 생성
blob=pd.DataFrame()

#while문을 돌리기위해 i생성(시작점)
i=1000

#광고영상이 대신 다운받아지는것을 방지하기 위해 광고일 경우 50초(광고시간)를 기다렸다가 진행
t=0

#i가 df의 길이 만큼 반복
while i < len(df):

    #df의 site칼럼에서 url받기
    baseUrl = df['site'][i]
    #driver실행
    driver.get(baseUrl)
    soup = BeautifulSoup(driver.page_source, "html")
    #실행된 site에서 영상 부분 태그
    v_url=soup.find('iframe',id='player_vod')['src']
    #주소에서 화질 변경 HIGH는 720 HIGH4는 1080
    v_url=v_url.replace('HIGH','HIGH4')
    #driver실행
    driver.get(v_url)
    #바로 src를 얻어오면 src가 생성이 되어있지않기 때문에 src가 생성될 수 있게 대기
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, "html")
    
    #광고영상 url엔 creative가 들어가기 때문에 creative가 들어있을 경우 다시 반복
    if 'creative' in soup.find('video',id='videoElement')['src']:
        continue
    #src가 blob으로 되어있는 경우 blob df에 넣기 
    if 'blob' in soup.find('video',id='videoElement')['src']:
        blob=blob.append(df.loc[i])
    else:
        #print(soup.find('video',id='videoElement')['src'])
        start=time.time()
        print('start: ',start)
        #파일 다운받기 형식은 날짜+팀+dh+년도+영상번호.mp4
        urllib.request.urlretrieve(soup.find('video',id='videoElement')['src'],df['date'][i]+df['team'][i]+df['dh'][i]+'2021-'+df['number'][i]+'.mp4')
        end=time.time()
        print('end: ',end)
        print(end-start)
    i+=1

In [ ]:
#시작점 끝점 직접입력

#driver생성
driver = webdriver.Chrome("c:/data/chromedriver")#, options=chrome_options)

#src가 블롭형식으로 되어있을 때 그 리스트를 받기 위해서 생성
blob=pd.DataFrame()

#while문을 돌리기위해 i생성(시작점) 직접입력받음
i=int(input("시작점"))

last=int(input("끝점"))
#광고영상이 대신 다운받아지는것을 방지하기 위해 광고일 경우 50초(광고시간)를 기다렸다가 진행
t=0

#i가 df의 길이 만큼 반복
while i < last+1:
    print(i)
    #df의 site칼럼에서 url받기
    baseUrl = df['site'][i]
    #driver실행
    driver.get(baseUrl)
    soup = BeautifulSoup(driver.page_source, "html")
    #실행된 site에서 영상 부분 태그
    v_url=soup.find('iframe',id='player_vod')['src']
    #주소에서 화질 변경 HIGH는 720 HIGH4는 1080
    v_url=v_url.replace('HIGH','HIGH4')
    #driver실행
    driver.get(v_url)
    #바로 src를 얻어오면 src가 생성이 되어있지않기 때문에 src가 생성될 수 있게 대기
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, "html")  
    #광고영상 url엔 creative가 들어가기 때문에 creative가 들어있을 경우 다시 반복
    if 'creative' in soup.find('video',id='videoElement')['src']:
        continue
    #src가 blob으로 되어있는 경우 blob df에 넣기 
    if 'blob' in soup.find('video',id='videoElement')['src']:
        blob=blob.append(df.loc[i])
    else:
        #print(soup.find('video',id='videoElement')['src'])
        start=time.time()
        print('start: ',start)
        #파일 다운받기 형식은 날짜+팀+dh+년도+영상번호.mp4
        urllib.request.urlretrieve(soup.find('video',id='videoElement')['src'],'c:/data/'+df['date'][i]+df['team'][i]+df['dh'][i]+'2021-'+df['number'][i]+'.mp4')
        end=time.time()
        print('end: ',end)
        print(end-start)
    i+=1

In [ ]:
#이미 다운 받아놓은 영상 중 광고영상이 있을 경우 다시 다운받게 해주는 코드

driver = webdriver.Chrome("c:/data/chromedriver")
#src가 블롭형식으로 되어있을 때 그 리스트를 받기 위해서 생성
blob=pd.DataFrame()
#i를 0으로 초기화 시작점
i=0
#임의로 201로 지정해놓았지만 변경가능 끝점
while i < 201:
    #6메가가 안넘을 경우 광고이기 때문에 50초를 기다렸다가 제대로된 src가 나오면 다운받기
    if os.path.getsize(df['date'][i]+df['team'][i]+df['dh'][i]+'2021-'+df['number'][i]+'.mp4')<6000000:
        baseUrl = df['site'][i]
        driver.get(baseUrl)
        soup = BeautifulSoup(driver.page_source, "html")
        v_url=soup.find('iframe',id='player_vod')['src']
        v_url=v_url.replace('HIGH','HIGH4')
        driver.get(v_url)
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, "html")  
        time.sleep(50) 
        if 'blob' in soup.find('video',id='videoElement')['src']:
            blob=blob.append(df.loc[i])
        else:
            print(soup.find('video',id='videoElement')['src'])
            start=time.time()
            print('start: ',start)
            urllib.request.urlretrieve(soup.find('video',id='videoElement')['src'],df['date'][i]+df['team'][i]+df['dh'][i]+'2021-'+df['number'][i]+'.mp4')
            end=time.time()
            print('end: ',end)
            print(end-start)
    #i를 1증가시킨후 다시 반복
    i+=1

In [ ]:
#다운이 실행된 영상 중 오류로 안받아 졌거나 광고가 다운 받아져 있는 경우
#while문을 돌리기위해 i생성(시작점) 직접입력받음
i=int(input("시작점"))

last=int(input("끝점"))


driver = webdriver.Chrome("c:/data/chromedriver")

#src가 블롭형식으로 되어있을 때 그 리스트를 받기 위해서 생성
blob=pd.DataFrame()

while i < last+1:
    if os.path.